<h1> DattoRMM / N-Able comparison </h1>

In [ ]:
#data conditioning
import pandas as pd
import re
import datetime as dt

# data import and file manipulation
import os
import json

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns


# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports'

# dictionary location
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'


# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# Import N-Able Assets

In [ ]:
df_nable = pd.read_csv('.csv',skiprows=3)

# Data Shaping

## Standardize Column Names

In [ ]:
df = pd.read_csv(f'{dictionary_dir}.csv')
column_rename_dict = {}
for index, row in df.iterrows():
    currentColumn = row['currentColumn']
    standardColumn = row['standardColumn']
    column_rename_dict[currentColumn] = standardColumn

In [ ]:
def column_names(string):
    for k, v in column_rename_dict.items():
        try:
            result = re.sub(k.lower(), v, string.lower())
            # print(f'comparing {k.lower()} with {string.lower()}')
            if result != string.lower():
                print(f'Keyword found: {k}')
                print(f'Replacement value: {v}')
                print('\n')
                return v
                break
            elif k.lower() == string.lower():
                print(f'Keyword found: {k}')
                print(f'Replacement value: {v}')
                print('\n')
                return v
                break
        except Exception as e:
            print(e)
            break
    return string

In [ ]:
for column in list(df_nable.columns):
    result = column_names(column)
    df_nable.rename({column:result},axis=1,inplace=True)

## Standardize Client Names

In [ ]:
cu_dict = {'Federal Credit Union':'FCU','Credit Union':'CU'}
def reword_creditunion(string):
    for k, v in cu_dict.items():
        result = re.sub(k,v,string)
        if result != string:
            return result
            break
    return string

In [ ]:
def one_off_rename_cu(string):
    if string == 'Example CU':
        return 'Example 1st CU'
    elif string == 'Golden Rule CU':
        return '[REDACTED]'
    else:
        return string

In [ ]:
df_nable['clientName'] = df_nable['clientName'].apply(reword_creditunion)
df_nable['clientName'] = df_nable['clientName'].apply(one_off_rename_cu)

In [ ]:
df_nable = df_nable[~df_nable['deviceType'].isin(['Servers - Linux','Storage','Switch/Router','Servers - ESXi','Other'])]

In [ ]:
df_nable['hostname'] = df_nable['hostname'].apply(lambda x: x.upper())

In [ ]:
df_nable['deviceType'].unique()

In [ ]:
df_nable['deviceType'].unique()

In [ ]:
total_value_counts = df_nable['clientName'].value_counts()

In [ ]:
total_value_counts

### Create 360 Client List

In [ ]:
nable_client_list = list(df_nable['clientName'].unique())

# Create DattoRMM Assets DataFrame

In [ ]:
# import and assign secrets from env.ini

dattormm_config = config['dattormm']

base_uri = dattormm_config['base_uri']

## Create auth token

In [ ]:
# call token api url
token_uri = f'{base_uri}/auth/oauth/token'


# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = dattormm_config['api_key']
data['password'] = dattormm_config['api_secret']

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

## Create DataFrame via API Call Iteration


In [ ]:

## Create Devices DataFrame
# request content response
request_url = f'{base_uri}/api/v2/account/devices'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_dattormm = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    df_current_page = pd.DataFrame(c_dict['devices'])
    df_dattormm = pd.concat([df_dattormm, df_current_page], ignore_index=False)

# Data Shaping

## Create New Columns from Dictionary Columns

### Set Index to device UID

In [ ]:
df_dattormm.set_index('uid',inplace=True)

### Type | Category

In [ ]:
def device_category(device):
    if device == None:
        return None
    else:
        return device['category']

In [ ]:
def device_type(device):
    if device == None:
        return None
    else:
        return device['type']

In [ ]:
df_dattormm['category'] = df_dattormm['deviceType'].apply(device_category)
df_dattormm['type'] = df_dattormm['deviceType'].apply(device_type)

# Rename 'type' values to split devices into (2) : 'computer' or 'server'
#df_dattormm['type'].replace({'Desktop':'computer','Laptop':'computer','Server':'server'},inplace=True)

In [ ]:
df_dattormm.drop(columns='deviceType',inplace=True)

In [ ]:
df_dattormm['hostname'] = df_dattormm['hostname'].apply(lambda x: x.upper())

# Compare DataFrames

In [ ]:
df_dattormm_compare = df_dattormm[df_dattormm['siteName'].isin(nable_client_list)]
df_dattormm_compare = df_dattormm_compare[['siteName','hostname','intIpAddress']]

In [ ]:
df_nable = df_nable.drop('siteName',axis=1).rename({'clientName':'siteName'},axis=1)
df_nable_compare = df_nable[['siteName','hostname','intIpAddress','deviceType']]

In [ ]:
df_dattormm_compare

In [ ]:
df_nable_compare['siteName'].unique()

## Compare DataFrames

In [ ]:
df_diff = pd.concat([df_dattormm_compare,df_nable_compare]).drop_duplicates(subset=(['siteName','hostname']),ignore_index=True,keep=False)

In [ ]:
df_diff.to_csv(export_folder + "/nable_onboard_tracking/" +  '/datto_nable_diff_' +str(current_time) + '.csv',index=False)

In [ ]:
def add_diff_flag(inputRow):
    row_dict = dict(inputRow)
    is_diff = 0
    for index,row in df_diff.iterrows():
        if (row['siteName'] == inputRow['siteName']) & (row['hostname'] == inputRow['hostname']) & (row['intIpAddress'] == inputRow['intIpAddress']):
            is_diff = 1
            break
        else:
            pass
    row_dict['notInDatto'] = is_diff
    return row_dict

In [ ]:
reconstruct_list = []
for index, row in df_nable_compare.iterrows():
    reconstruct_list.append(add_diff_flag(row))

In [ ]:
df_reconstruted = pd.DataFrame(reconstruct_list)

In [ ]:
df_reconstruted[df_reconstruted['notInDatto'] == 1]

In [ ]:
sns.set_style('white')
fig = plt.figure(figsize=(80,20))

chart = sns.countplot(
    data=df_reconstruted[['siteName','deviceType','notInDatto']].sort_values('siteName'),
    x='siteName',
    hue='notInDatto',
    palette=({1:'red',0:'green'})
)

chart.set_xticklabels(chart.get_xticklabels(),
    rotation=45,
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large'
)

plt.xlabel('Client Sites',fontsize=40)
plt.ylabel('Counts',fontsize=40)

plt.title('DattoRMM Migration from N-Able',fontsize=80)

fig.savefig(export_folder + "/nable_onboard_tracking/" + 'datto_nable_diff_' + str(current_time) + '.png')

In [ ]:
tracking_csv_list = []
for root, dirs, files in (os.walk(f'{export_folder}/nable_onboard_tracking/')):
    for file in files:
        if re.match(r'.*.csv$',file):
            tracking_csv_list.append(f'{root}{file}')

In [ ]:
def fix_neg_percent(num):
    if num < 0:
        return 0
    else:
        return num

In [ ]:
def create_time_dict(row):
    time_dict = {}
    time_dict['siteName'] = (row['index'])
    time_dict['percentComplete'] = (row['percentComplete'])
    time_dict['timestamp'] = (row['timestamp'])

    return time_dict

In [ ]:
time_series_row_list = []

for file in tracking_csv_list[:]:
    try:
        df = pd.read_csv(file)
    except:
        df = pd.read_csv(file,'utf-16')

    diff_value_counts = df['siteName'].value_counts()
    df_variance = pd.concat([total_value_counts,diff_value_counts],axis=1)
    df_variance.rename({'clientName':'totalAssets','siteName':'notInDattoRMM'},axis=1,inplace=True)
    df_variance['percentComplete'] = round(((df_variance['totalAssets'] - df_variance['notInDattoRMM']) / df_variance['totalAssets']) * 100)
    df_variance['percentComplete'] = df_variance['percentComplete'].apply(fix_neg_percent)
    timestamp = dt.datetime.utcfromtimestamp(os.path.getmtime(file)).strftime('%Y/%m/%d %H:%M:%S')
    df_variance = df_variance.fillna(0).astype('int')
    df_variance['timestamp'] = timestamp

    for index,row in df_variance.reset_index().iterrows():
        time_series_row_list.append(create_time_dict(row))

In [ ]:
time_series_row_list

In [ ]:
tracking_csv_list

In [ ]:
df = pd.DataFrame(time_series_row_list)

In [ ]:
df

In [ ]:
fig = plt.figure(figsize=(60,50))
sns.lineplot(data=df[df['siteName'] == '[REDACTED]'].sort_values('siteName',ascending=True),y='percentComplete',x='timestamp',hue='siteName')
fig.savefig('.png')

In [ ]:
df_nable_compare[df_nable_compare['hostname'].str.contains('RMM')].to_csv(f'{export_folder}.csv',index=False)